In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

24/03/16 15:46:45 WARN Utils: Your hostname, cenviiBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.102 instead (on interface en0)
24/03/16 15:46:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/16 15:46:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# %%capture
# !curl -LO https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

In [4]:
# !gunzip -f fhvhv_tripdata_2021-01.csv.gz

In [5]:
!wc -l fhvhv_tripdata_2021-01.csv

 11908469 fhvhv_tripdata_2021-01.csv


In [6]:
df = spark.read.option("header", "true").csv("fhvhv_tripdata_2021-01.csv")

In [7]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:55:19', dropoff_datetime='2021-01-01 01:18:21', PULocationID='152', DOLocationID='167', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:23:56', dropoff_datetime='2021-01-01 00:38:05', PULocationID='233', DOLocationID='142', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:42:51', dropoff_datetime='2021-01-01 00:45:50', PULocationID='142', DOLocationID='143', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:48:14', dropoff_datetime='2021-01-01 01:08:42', PULocationID='143', DOLocationID='78', SR_Flag=None)]

In [8]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [9]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [10]:
!head -n 10 head.csv

hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,
HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,
HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,
HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,
HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,
HV0005,B02510,2021-01-01 00:06:59,2021-01-01 00:43:01,88,42,
HV0005,B02510,2021-01-01 00:50:00,2021-01-01 01:04:57,42,151,
HV0003,B02764,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226,
HV0003,B02875,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255,


In [11]:
!wc -l head.csv

    1001 head.csv


In [12]:
import pandas as pd

In [13]:
df_pandas = pd.read_csv("head.csv")

In [14]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [15]:
spark.createDataFrame(df_pandas).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|    NaN|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|    NaN|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|    NaN|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|    NaN|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|    NaN|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [16]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [17]:
from pyspark.sql import types

In [18]:
schema = types.StructType(
    [
        types.StructField("hvfhs_license_num",
                          types.StringType(), nullable=True),
        types.StructField("dispatching_base_num",
                          types.StringType(), nullable=True),
        types.StructField("pickup_datetime",
                          types.TimestampType(), nullable=True),
        types.StructField("dropoff_datetime",
                          types.TimestampType(), nullable=True),
        types.StructField("PULocationID", types.IntegerType(), nullable=True),
        types.StructField("DOLocationID", types.IntegerType(), nullable=True),
        types.StructField("SR_Flag", types.StringType(), nullable=True),
    ]
)

In [19]:
df = (
    spark.read.option(
        "header",
        "true",
    )
    .schema(schema)
    .csv("fhvhv_tripdata_2021-01.csv")
)

In [20]:
df.head(10)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [21]:
df = df.repartition(24)

In [22]:
df.write.parquet("fhvhv/2021/01/", mode="overwrite")

24/03/16 15:46:58 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
24/03/16 15:47:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/16 15:47:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/16 15:47:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/03/16 15:47:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/16 15:47:01 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Sca

In [23]:
df = spark.read.parquet("fhvhv/2021/01/")
df

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string]

In [24]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02880|2021-01-03 01:03:21|2021-01-03 01:14:58|         216|          38|   NULL|
|           HV0005|              B02510|2021-01-03 23:19:19|2021-01-03 23:27:32|          63|         177|   NULL|
|           HV0005|              B02510|2021-01-03 17:40:49|2021-01-03 18:01:41|          82|          28|   NULL|
|           HV0003|              B02875|2021-01-04 09:58:02|2021-01-04 10:13:18|          60|         242|   NULL|
|           HV0003|              B02867|2021-01-01 02:37:46|2021-01-01 02:48:17|          28|         130|   NULL|
|           HV0003|              B02884|2021-01-01 02:14:10|2021-01-01 02:24:55|

In [25]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [26]:
!head -n 10 head.csv

hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,
HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,
HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,
HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,
HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,
HV0005,B02510,2021-01-01 00:06:59,2021-01-01 00:43:01,88,42,
HV0005,B02510,2021-01-01 00:50:00,2021-01-01 01:04:57,42,151,
HV0003,B02764,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226,
HV0003,B02875,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255,


In [27]:
df.select(
    "pickup_datetime",
    "dropoff_datetime",
    "PULocationID",
    "DOLocationID",
).filter(df.hvfhs_license_num == "HV0003").show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-03 01:03:21|2021-01-03 01:14:58|         216|          38|
|2021-01-04 09:58:02|2021-01-04 10:13:18|          60|         242|
|2021-01-01 02:37:46|2021-01-01 02:48:17|          28|         130|
|2021-01-01 02:14:10|2021-01-01 02:24:55|         239|         262|
|2021-01-02 20:20:37|2021-01-02 20:28:17|          26|         227|
|2021-01-03 15:43:48|2021-01-03 15:56:31|          62|          49|
|2021-01-02 08:20:31|2021-01-02 08:28:55|         181|         188|
|2021-01-01 01:11:14|2021-01-01 01:15:55|          69|          69|
|2021-01-02 06:33:55|2021-01-02 06:47:42|          81|         174|
|2021-01-03 17:43:58|2021-01-03 17:50:50|         255|         256|
|2021-01-02 12:26:08|2021-01-02 12:31:39|         240|         174|
|2021-01-03 20:13:29|2021-01-03 20:39:19|       

In [28]:
df.select(
    "pickup_datetime",
    "dropoff_datetime",
    "PULocationID",
    "DOLocationID",
).filter(df.hvfhs_license_num == "HV0003").take(5)

[Row(pickup_datetime=datetime.datetime(2021, 1, 3, 1, 3, 21), dropoff_datetime=datetime.datetime(2021, 1, 3, 1, 14, 58), PULocationID=216, DOLocationID=38),
 Row(pickup_datetime=datetime.datetime(2021, 1, 4, 9, 58, 2), dropoff_datetime=datetime.datetime(2021, 1, 4, 10, 13, 18), PULocationID=60, DOLocationID=242),
 Row(pickup_datetime=datetime.datetime(2021, 1, 1, 2, 37, 46), dropoff_datetime=datetime.datetime(2021, 1, 1, 2, 48, 17), PULocationID=28, DOLocationID=130),
 Row(pickup_datetime=datetime.datetime(2021, 1, 1, 2, 14, 10), dropoff_datetime=datetime.datetime(2021, 1, 1, 2, 24, 55), PULocationID=239, DOLocationID=262),
 Row(pickup_datetime=datetime.datetime(2021, 1, 2, 20, 20, 37), dropoff_datetime=datetime.datetime(2021, 1, 2, 20, 28, 17), PULocationID=26, DOLocationID=227)]

In [29]:
from pyspark.sql import functions as F

In [30]:
df.withColumn(
    "pickup_date",
    F.to_date(df.pickup_datetime),
).withColumn(
    "dropoff_date",
    F.to_date(df.dropoff_datetime),
).select(
    "pickup_date",
    "dropoff_date",
    "PULocationID",
    "DOLocationID",
).show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-03|  2021-01-03|         216|          38|
| 2021-01-03|  2021-01-03|          63|         177|
| 2021-01-03|  2021-01-03|          82|          28|
| 2021-01-04|  2021-01-04|          60|         242|
| 2021-01-01|  2021-01-01|          28|         130|
| 2021-01-01|  2021-01-01|         239|         262|
| 2021-01-02|  2021-01-02|          26|         227|
| 2021-01-04|  2021-01-04|          10|         226|
| 2021-01-03|  2021-01-03|          62|          49|
| 2021-01-02|  2021-01-02|         181|         188|
| 2021-01-01|  2021-01-01|          69|          69|
| 2021-01-03|  2021-01-03|          79|         261|
| 2021-01-01|  2021-01-01|         222|          77|
| 2021-01-02|  2021-01-02|          81|         174|
| 2021-01-03|  2021-01-03|         255|         256|
| 2021-01-02|  2021-01-02|         240|       

In [31]:
def crazy_stuff(base_num: str) -> str:
    num = int(base_num[1:])
    if num % 7 == 0:
        return f"s/{num:03x}"
    if num % 3 == 0:
        return f"a/{num:03x}"
    return f"e/{num:03x}"

In [32]:
crazy_stuff("B02884")

's/b44'

In [33]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [34]:
df.withColumn(
    "pickup_date",
    F.to_date(df.pickup_datetime),
).withColumn(
    "dropoff_date",
    F.to_date(df.dropoff_datetime),
).withColumn("base_id", crazy_stuff_udf(df.dispatching_base_num)).select(
    "base_id",
    "pickup_date",
    "dropoff_date",
    "PULocationID",
    "DOLocationID",
).show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  a/b40| 2021-01-03|  2021-01-03|         216|          38|
|  e/9ce| 2021-01-03|  2021-01-03|          63|         177|
|  e/9ce| 2021-01-03|  2021-01-03|          82|          28|
|  e/b3b| 2021-01-04|  2021-01-04|          60|         242|
|  e/b33| 2021-01-01|  2021-01-01|          28|         130|
|  s/b44| 2021-01-01|  2021-01-01|         239|         262|
|  e/b32| 2021-01-02|  2021-01-02|          26|         227|
|  e/9ce| 2021-01-04|  2021-01-04|          10|         226|
|  e/a39| 2021-01-03|  2021-01-03|          62|          49|
|  e/b35| 2021-01-02|  2021-01-02|         181|         188|
|  s/b3d| 2021-01-01|  2021-01-01|          69|          69|
|  e/9ce| 2021-01-03|  2021-01-03|          79|         261|
|  e/9ce| 2021-01-01|  2021-01-01|         222|          77|
|  e/b3e| 2021-01-02|  2